<a href="https://colab.research.google.com/github/ARMargolis/matrix_bin_packing/blob/main/Matrix_Bin_Packing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Matrix Multiplication for Efficient Bin Packing
In this workbook, I develop a new method for efficient bin packing using matrix multiplication. Bin packing is one of many optimization problems with real world applications. There are heuristic algorithms that come within a few percentage points of the optimal answer, but for many business use cases at scale, that small percentage can mean a large loss of value.
I take an approach of using high-memory to efficiently test a large number of combinations to find a better solution than standard heuristic approaches. The first version uses numpy, but future versions will use GPUs and TPUs.


Example problems from

 https://people.sc.fsu.edu/~jburkardt/datasets/bin_packing/bin_packing.html (0 to 3)
https://scipbook.readthedocs.io/en/latest/bpp.html (4)
http://www.or.deis.unibo.it/staff_pages/martello/Slides_Estoril_Martello.pdf (

In [1]:
bin_problems=[(100, [70, 60, 50, 33, 33, 33, 11, 7, 3]),
              (100, [99, 94, 79, 64, 50, 46, 43, 37, 32, 19, 18, 7, 6, 3]),
              (100, [49, 41, 34, 33, 29, 26, 26, 22, 20, 19]),
              (524, [442, 252, 252, 252, 252, 252, 252, 252, 127, 127, 127, 127, 127, 106,
                   106, 106, 106, 85, 84, 46, 37, 37, 12, 12, 12, 10, 10, 10, 10, 10, 10, 9]),
              (9, [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]),
              (100, [99, 94, 90, 88, 80, 10, 10, 6, 5, 5, 4, 4]),
              (100, [49, 41, 34, 33, 29, 26, 26, 22, 20, 19])]

In [2]:
import time, numpy as np
import itertools
from math import factorial

def binpack(bin_cap, articles, noisy=False, limit=0, threshold=.995):
  arts2sort=np.sort(np.array(articles),kind='mergesort')
  bin_list=[]
  cycles=0
  while arts2sort.sum()>bin_cap and (limit>cycles or limit==0):
    cycles+=1
    #Max_items: What is the most items you can fit (by starting with the smallest and counting up)
    #Min_items: Start with the top and count down
    max_items=np.where(np.cumsum(arts2sort)>bin_cap)[-1][0]
    min_items=np.where(np.cumsum(np.flip(arts2sort))>bin_cap)[-1][0]
    if noisy:
      print(arts2sort, '\nMin:', min_items, 'Max:', max_items)
    
    #If you can fit more of some items than others, try different amounts until
    if max_items>min_items:
      max_pack=[np.concatenate([np.zeros(arts2sort.shape[0]-min_items, dtype=bool), np.ones(min_items, dtype=bool)])]
      max_load=[arts2sort[-min_items:].sum()]
      m=min_items+1
      #Starting with min_items, try every amount until you get a size that fits within threshold
      while m<=max_items and max_load[-1]<bin_cap*threshold:
        #Make every combination of m items from arts2sort
        num_combos=int(factorial(arts2sort.shape[0])/(factorial(arts2sort.shape[0]-m)*factorial(m)))
        combos=list(itertools.combinations(range(arts2sort.shape[0]), m))
        combos_tup=[tuple(itertools.chain.from_iterable([n]*m for n in range(num_combos))),
            tuple(itertools.chain.from_iterable(combos))]

        #Make a matrix of m ones and the rest zeros, with the ones being every combo
        bin_matrix=np.zeros((num_combos, arts2sort.shape[0]), dtype=bool)
        bin_matrix[combos_tup]=1

        #Find out how much load they carry, then sort them and choose the highest load that fits in a bin
        pack_opts=np.matmul(bin_matrix, arts2sort)
        pack_order=pack_opts.argsort(kind='mergesort')
        best_opt=np.where(pack_opts[pack_order]<=bin_cap)[-1][-1]
        max_pack.append(bin_matrix[pack_order[best_opt]])
        max_load.append(pack_opts[pack_order[best_opt]])
        m+=1
        if noisy:
          print("best_opt:",best_opt,"load:",pack_opts[pack_order[best_opt]],
                "used:", arts2sort[bin_matrix[pack_order[best_opt]]])
      if noisy:
        print('max_load:', max_load, max(max_load))
      #After trying different numbers of items, choose the best one and the best combination with that many items
      best_m=max_load.index(max(max_load))
      arts2add=np.arange(arts2sort.shape[0])[max_pack[best_m]]
      if noisy:
        print('best_m:', best_m, 'best_load:', max_load[best_m], max_pack[best_m], '\narts2add:', arts2add)

      #Add those items to the bin_list and remove from arts2sort
      bin_list.append(arts2sort[arts2add])
      arts2sort=np.delete(arts2sort,arts2add)

    #If you can only fit one amount of items, just choose the largest items
    else:
      bin_list.append(arts2sort[-max_items:])
      arts2sort=np.delete(arts2sort,np.arange(arts2sort.shape[0]-max_items,arts2sort.shape[0]))

  return bin_list+[arts2sort]

Now let's test it for performance!

In [3]:
for (cap, articles) in bin_problems:
  print(time.ctime())
  print(binpack(cap, articles, noisy=True), '\n')
print('Done:', time.ctime())

Tue Mar 30 10:18:39 2021
[ 3  7 11 33 33 33 50 60 70] 
Min: 1 Max: 5
best_opt: 29 load: 93 used: [33 60]
best_opt: 43 load: 100 used: [ 7 33 60]
max_load: [70, 93, 100] 100
best_m: 2 best_load: 100 [False  True False False False  True False  True False] 
arts2add: [1 5 7]
[ 3 11 33 33 50 70] 
Min: 1 Max: 4
best_opt: 11 load: 83 used: [33 50]
best_opt: 9 load: 94 used: [11 33 50]
best_opt: 2 load: 97 used: [ 3 11 33 50]
max_load: [70, 83, 94, 97] 97
best_m: 3 best_load: 97 [ True  True False  True  True False] 
arts2add: [0 1 3 4]
[33 70] 
Min: 1 Max: 1
[array([ 7, 33, 60]), array([ 3, 11, 33, 50]), array([70]), array([33])] 

Tue Mar 30 10:18:39 2021
[ 3  6  7 18 19 32 37 43 46 50 64 79 94 99] 
Min: 1 Max: 6
best_opt: 57 load: 100 used: [ 6 94]
max_load: [99, 100] 100
best_m: 1 best_load: 100 [False  True False False False False False False False False False False
  True False] 
arts2add: [ 1 12]
[ 3  7 18 19 32 37 43 46 50 64 79 99] 
Min: 1 Max: 5
best_opt: 44 load: 98 used: [19 79]
b

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


best_opt: 3682 load: 522 used: [ 37 106 127 252]
max_load: [442, 488, 516, 522] 522
best_m: 3 best_load: 522 [False False False False False  True False False False False False False
  True False False False False  True  True False] 
arts2add: [ 5 12 17 18]
[  9  12  12  12  37  46  84  85 106 106 106 127 127 127 127 442] 
Min: 1 Max: 10
best_opt: 110 load: 488 used: [ 46 442]
best_opt: 468 load: 500 used: [ 12  46 442]
best_opt: 1380 load: 512 used: [ 12  12  46 442]
best_opt: 2884 load: 524 used: [ 37 106 127 127 127]
max_load: [442, 488, 500, 512, 524] 524
best_m: 4 best_load: 524 [False False False False  True False False False False False  True False
  True  True  True False] 
arts2add: [ 4 10 12 13 14]
[  9  12  12  12  46  84  85 106 106 127 442] 
Min: 1 Max: 9
best_opt: 49 load: 488 used: [ 46 442]
best_opt: 129 load: 500 used: [ 12  46 442]
best_opt: 219 load: 512 used: [ 12  12  46 442]
best_opt: 256 load: 524 used: [ 12  12  12  46 442]
max_load: [442, 488, 500, 512, 524] 524